In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Sequential
from keras import layers
from keras.backend import clear_session
from keras.preprocessing.text import Tokenizer
from keras.layers import Activation, Dense, Reshape 
import numpy as np
from keras import backend as K
import matplotlib.pyplot as plt
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout, Conv1D, GlobalMaxPooling1D
from keras.models import load_model
from numpy import asarray
from numpy import save
from numpy import load
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import esprima as esp
import re
from keras.initializers import Constant
from tensorflow.keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
from tensorflow.keras.utils import plot_model
from keras import Model
from tensorflow.keras.utils import Sequence
import pydot
import gensim 
from nltk.tokenize import word_tokenize
import pickle
from keras import backend as K
import os
import random
import re
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import logging
from time import time
from functools import partial
# from concurrent.futures import ProcessPoolExecutor
import concurrent.futures
from threading import current_thread
# from tqdm.notebook import tqdm
from tqdm import tqdm
import gc
# Set the random seed for reproducible results
np.random.seed(123)
plt.style.use('ggplot')

In [2]:
# train_batch_size = 50
# test_batch_size = int(test_df.shape[0]/(train_df.shape[0]*train_batch_size))
# validate_batch_size = int(validation_df.shape[0]/(train_df.shape[0]*train_batch_size))
# num_words = 400000
num_words = 400
# maxlen = 1024
maxlen = 100
embedding_dim = 50
myoptimizer = 'adam'
myloss= 'binary_crossentropy'
mymetrics = 'accuracy'

In [3]:
# from google.colab import drive
# drive.mount("/content/drive")

In [4]:
def save_object(obj, filename):
    filename = os.path.join('G:\JSContanaDataSet\par',filename)
    with open(filename+".pkl", 'wb') as outp:
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)
    outp.close()

In [5]:
def load_object(filename):
    filename = os.path.join('G:\JSContanaDataSet\par',filename)
    with open(filename+".pkl", 'rb') as outp:
        loaded_object = pickle.load(outp)
    outp.close()
    return loaded_object

In [6]:
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [7]:
def lowering_remove_url_ip_(script):
#     print("=====")
    script = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', script)
    script = re.sub(r'((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)', '', script)
    script = re.sub(r"(/\*([^*]|[\r\n]|(\*+([^*/]|[\r\n])))*\*+/)|(//.*)",'',script)
    script = script.lower()
    return script

In [8]:
def preprocessing_dataset_(scripts,fit = True):
    if fit:
        tokenizer = Tokenizer(num_words=num_words)
        tokenizer.fit_on_texts(scripts)
        our_tokenizer = save_object(tokenizer, "our_tokenizer")
    else :
        tokenizer = load_object("our_tokenizer")
    X = tokenizer.texts_to_sequences(scripts)
    if fit:
        vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
        save_object(vocab_size, "vocab_size")
    X = pad_sequences(X, padding='post', maxlen=maxlen)
    return X

In [9]:
# convert java script code to the abstract syntax tree
# then extract the sequences of syntax units with detailed information
def sequence_of_syntax_units(scripts):
    """
    sctipts : list or single javascript code 
    return sequence of syntax units
    each syntax unit correspond to line in an abstract syntax tree
    """
    if type(scripts) is list:
        # esp.parseScript(script) returns abstract syntax tree of each js scripts
        return [re.sub('\s+', ' ', ''.join(str(esp.parseScript(script)).split('\n'))) for script in scripts]
    elif type(scripts) is str:
        return re.sub('\s+', ' ', ''.join(str(esp.parseScript(scripts)).split('\n')))
    else:
        raise ValueError('The type of scripts parameter must be {list or string}')

In [10]:
# create word tokens
def sequence_of_syntax_units_to_tokens(seq_of_syn_units):
    """
    Convert Sequence of syntax units to tokens to train  word2vec model 
    -----------------------------------------
    seq_of_syn_units: list of sequence syntax units correspond to abstract syntax trees
    """
    return list(map(word_tokenize, seq_of_syn_units))

In [11]:
class CustomDataGen(Sequence):
    
    def __init__(self, df, X_col, y_col,batch_size,fit=False,shuffle=True,parent_path=""):
        
        self.df = df.copy()
        self.X_col = X_col
        self.y_col = y_col
        self.batch_size = batch_size
        self.fit = fit
        self.shuffle = shuffle
        self.parent_path = parent_path
        self.n = df.shape[0]
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
    
    def __get_input(self, path):
        try:
            with open(os.path.join(self.parent_path,path), 'r', encoding='utf-8') as js_file:
                script = js_file.read()
            script = lowering_remove_url_ip_(script)
            script = sequence_of_syntax_units(script)
            return script
        except:
            return 'None'
    def __get_output(self, label):
        return label
#         return keras.utils.to_categorical(label, num_classes=2)
    
    def __get_data(self, batches):

        path_batch = batches[self.X_col['path']]
        
        name_batch = batches[self.y_col['name']]

        X_batch = np.asarray([self.__get_input(x) for x in path_batch]).astype('object')
        
        y_batch = np.asarray([self.__get_output(y) for y in name_batch]).astype('object')
        
        index = np.where(X_batch=='None')[0]
        
        np.delete(X_batch, index, axis=0)
        np.delete(y_batch, index, axis=0)
        
        X_batch  = preprocessing_dataset_(X_batch , fit = self.fit )
        
        X_batch = K.cast_to_floatx(X_batch)
        y_batch = K.cast_to_floatx(y_batch)
        

        return X_batch, y_batch
    
    def __getitem__(self, index):
        
        batches = self.df[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__get_data(batches)        
        return X, y
    
    def __len__(self):
        return self.n // self.batch_size

In [12]:
def feature_selection(x_chunk,y_chunk):
    indexes = random.sample(range(3, x_chunk.shape[0]), random.randint(1, x_chunk.shape[0]-3))
    x_chunk[indexes,:,:] = 0
    y_chunk = y_chunk[indexes]
    return x_chunk,y_chunk
def feature_selection_(x_chunk):
    indexes = random.sample(range(3, x_chunk.shape[0]), random.randint(1, x_chunk.shape[0]-3))
    x_chunk[:,indexes,:] = 0
    return x_chunk
def feature_selection__(x_chunk):
    indexes = random.sample(range(3, x_chunk.shape[0]), random.randint(1, x_chunk.shape[0]-3))
    x_chunk[:,:,indexes] = 0
    return x_chunk

In [13]:
def evaluation_(model,history,X_train,y_train):
    train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=False)

In [14]:
def evaluation(model,chunk_nuber,history,X_train,y_train,X_test,y_test,model_comparison_table):
    train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=False)
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=False)
    predicted = model.predict(X_test)
    Classification_report = classification_report(np.round(model.predict(X_test)), y_test)
    inputer(model_comparison_table,chunk_nuber,train_loss, train_accuracy,test_loss, test_accuracy,str(Classification_report))

In [15]:
def create_model(shape=100,without_lstm=False):
    model = Sequential()
    if not without_lstm:
        model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
        model.add(Bidirectional(LSTM(units=50, input_shape=(None, 50), return_sequences=True)))
    else :
         model.add(layers.InputLayer(input_shape=shape))
        
    
    model.add(Bidirectional(LSTM(units=50, input_shape=(None, 50), return_sequences=True)))

    # TextCNN with 4 conv layers
    model.add(Conv1D(128, 7, activation='tanh', input_shape=(None, 32)))
    model.add(Conv1D(128, 15, activation='tanh'))
    model.add(Conv1D(128, 25, activation='tanh'))
    model.add(Conv1D(128, 35, activation='tanh'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=myoptimizer,
                  loss=myloss,
                  metrics=[mymetrics])
#     model.summary()
    return model

In [16]:
def fit_model_(model,X_train,y_train):
    history = model.fit(X_train, y_train,epochs=10,verbose=False)
    return model , history

In [17]:
def fit_model(model,X_train,y_train,X_validation,y_validation):
    history = model.fit(X_train, y_train,epochs=10,verbose=False,validation_data=(X_validation, y_validation))
    return model , history

In [18]:
def creat_metrics():
    model_comparison_table = {}
    model_comparison_table['chunk_nuber'] = []
    model_comparison_table['train_loss'] = []
    model_comparison_table['train_accuracy'] = []
    model_comparison_table['test_loss'] = []
    model_comparison_table['test_accuracy'] = []
    model_comparison_table['classification_report'] = []
    return model_comparison_table

In [19]:
def inputer(model_comparison_table,chunk_nuber,train_loss, train_accuracy,test_loss,test_accuracy,classification_report):#precision_score,recall_score,f1_score):
    model_comparison_table['chunk_nuber'].append(str(chunk_nuber))
    model_comparison_table['train_loss'].append(train_loss)
    model_comparison_table['train_accuracy'].append(train_accuracy)
    model_comparison_table['test_loss'].append(test_loss)    
    model_comparison_table['test_accuracy'].append(test_accuracy)  
    model_comparison_table['classification_report'].append(classification_report)    

In [20]:
# '/content/drive/MyDrive/Colab Notebooks/XSS_dataset.csv'
# df = pd.read_csv('G:\JSContanaDataSet\XSS_dataset.csv')
# df1 = df.sa1mple(2000)
# df1.to_csv('G:\JSContanaDataSet\XSS_dataset_Samle.csv',index=False)
# test_df = df.iloc[:int((df.shape[0]*20)/100)]
# validation_df = df.iloc[int((df.shape[0]*20)/100):int(2*((df.shape[0]*20)/100))]
# train_df = df.iloc[int(2*((df.shape[0]*20)/100)):]

In [23]:
def chunking_data(Data,validation_size=0.10,test_size=0.20,train_size=0.70):
    
    test_data = Data.iloc[:int((Data.shape[0]*test_size))]
    
    validation_data = Data.iloc[int((Data.shape[0]*test_size)):(int((Data.shape[0]*validation_size))+int((Data.shape[0]*test_size)))]
    
    train_data = Data.iloc[(int((Data.shape[0]*test_size))+int((Data.shape[0]*validation_size))):]
    
    X_test = test_data[test_data.columns[-2]]
    y_test = test_data[test_data.columns[-1]]
    
    X_validation = validation_data[validation_data.columns[-2]]
    y_validation = validation_data[validation_data.columns[-1]]
    
    X_train = train_data[train_data.columns[-2]]
    y_train = train_data[train_data.columns[-1]]
    
    return X_train , y_train , X_test , y_test , X_validation , y_validation

In [24]:
def read__script(script_paths,parent_path,y):
    x_temp = []
    y_temp = []
    todrop = []
    for path in tqdm(script_paths):
        try:
            with open(os.path.join(parent_path,path), 'r', encoding='utf-8') as js_file:
                script = js_file.read()
            js_file.close()
#             script = lowering_remove_url_ip_(script)
            script = sequence_of_syntax_units(script)
            new_y = y[script_paths.index(path)]
            x_temp.append(script)
            y_temp.append(new_y)
        except:
            pass
    if len(x_temp)>0:
        return x_temp , np.array(y_temp)
    else:
        return None , None

In [ ]:
model_comparison_table = creat_metrics()
model_is_created = False
i = 0 
for df_iterator in pd.read_csv("G:\JSContanaDataSet\XSS_dataset.csv",chunksize=2000):
    try:
        print("============================"+str(i)+"=================================")
        X_train , y_train , X_test , y_test , X_validation , y_validation = chunking_data(df_iterator)
        
        X_train , y_train = read__script(X_train.tolist(),"G:\JSContanaDataSet",y_train.tolist())
        X_test , y_test = read__script(X_test.tolist(),"G:\JSContanaDataSet",y_test.tolist())
        X_validation , y_validation = read__script(X_validation.tolist(),"G:\JSContanaDataSet",y_validation.tolist())
        
        
        if X_train!=None:
            X_train = preprocessing_dataset_(X_train , fit = True )
            X_test = preprocessing_dataset_(X_test , fit = False )
            X_validation = preprocessing_dataset_(X_validation , fit = False )


            print(X_train.shape)
            print(y_train.shape)

            print(X_test.shape)
            print(y_test.shape)

            print(X_validation.shape)
            print(y_validation.shape)

            if not model_is_created:
                vocab_size = load_object('vocab_size')
                train_test_Model = create_model()
                train_test_Model.save("train_test_Model.h5")
                model_is_created = True
                print(vocab_size)
            else:
                train_test_Model = load_model("train_test_Model.h5")

            train_test_Model , history_train_test_Model = fit_model(train_test_Model,X_train,y_train,X_validation,y_validation)

            evaluation(train_test_Model,i,history_train_test_Model,X_train,y_train,X_test,y_test,model_comparison_table)

            train_test_Model.save("train_test_Model.h5")
            i+=1
    except:
        print("error")

  1%|▌                                                                                | 9/1400 [00:00<00:19, 71.06it/s]

============================0=================================


100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:18<00:00, 10.70it/s]


(1350, 100)
(1350,)
(384, 100)
(384,)
(195, 100)
(195,)
80945


  0%|                                                                                         | 0/1400 [00:00<?, ?it/s]

...

  0%|                                                                                         | 0/1400 [00:00<?, ?it/s]

============================30=================================


100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [03:53<00:00,  1.17s/it]


error
============================30=================================


 36%|█████████████████████████████▌                                                   | 73/200 [02:34<04:03,  1.92s/it]

In [ ]:
save_object(model_comparison_table, "model_comparison_table")

In [ ]:
df = pd.DataFrame(model_comparison_table)
df.to_csv('G:/JSContanaDataSet/result_df.csv',index=False)
df.head(5)

In [ ]:
for i in df['classification_report']:
    print("================================")
    print(i)

In [54]:
len(X_train)

105

In [ ]:
for h in range(len(ss)):
    print("================"+str(h)+"=================")
    for i in ss[h]:
        print(i)

In [25]:
print(len(X_train))
print(X_train[0])

31
{ type: "Program", sourceType: "script", body: [ { type: "ExpressionStatement", expression: { type: "CallExpression", callee: { type: "Identifier", name: "eval" }, arguments: [ { type: "CallExpression", callee: { type: "MemberExpression", computed: False, object: { type: "CallExpression", callee: { type: "MemberExpression", computed: True, object: { type: "Identifier", name: "ac8Mz6M" }, property: { type: "Literal", value: "reverse", raw: "\"reverse\"" } }, arguments: [] }, property: { type: "Identifier", name: "join" } }, arguments: [ { type: "Literal", value: "", raw: "''" } ] } ] } } ]}


In [ ]:
for i, df_chunk in enumerate(df_iterator)

    print(df_chunk.shape)
    
    # Set writing mode to append after first chunk
    mode = 'w' if i == 0 else 'a'
    
    # Add header if it is the first chunk
    header = i == 0

    df_chunk.to_csv('G:\JSContanaDataSet\XSS_dataset.csv',
        index=False,  # Skip index column
        header=header, 
        mode=mode)

In [144]:
d = pd.read_csv(filepath_or_buffer = 'G:\JSContanaDataSet\XSS_dataset.csv',iterator=True,chunksize=1000)

In [146]:
for i in d:
    print(i['ScriptPath'])

1000    data/Farata/EnterpriseWebBook_sources/chapter7...
1001    data/alibaba/ots/benchmark/parse_getrow_result.js
1002                   data/mde/logan/lib/racer/runner.js
1003    data/regularex/FreePyBX/freepybx/public/js/doj...
1004    data/tshemsedinov/impress/examples/impressClou...
                              ...                        
1995    data/foursquare/foursquare-palmpre/Foursquare-...
1996    data/nedjo/openoutreach/profiles/openoutreach/...
1997    data/cdnjs/cdnjs/ajax/libs/yui/3.18.0/tree-sel...
1998                         data/yahoo/kobold/lib/cli.js
1999                     data/h5bp/html5please/lib/app.js
Name: ScriptPath, Length: 1000, dtype: object
2000    data/sebastienros/jint/Jint.Tests.Ecma/TestCas...
2001    data/city41/breakouts/breakouts/createjs/js/en...
2002    data/yahoo/mojito/tests/func/common/testpostpa...
2003    data/mozilla/rhino/testsrc/tests/ecma_2/Except...
2004    data/disnet/contracts.js/js/libs/mode/http/htt...
                          

30000    data/cdnjs/cdnjs/ajax/libs/yui/3.17.2/datatype...
30001    data/cakesolutions/the-pragmatic-haskeller/09-...
30002    data/Chorus/chorus/app/assets/javascripts/view...
30003      data/mout/mout/tests/spec/object/spec-filter.js
30004    data/ceph/calamari-clients/admin/app/scripts/a...
                               ...                        
30995    data/Paxa/postbird/node_modules/pg/node_module...
30996    data/ezsystems/PlatformUIBundle/Resources/publ...
30997    data/jbasdf/muck/public/javascripts/tiny_mce/p...
30998    data/juanpicado/marionettejs-sample-app/src/ap...
30999    data/cgravolet/scroblr/src/js/plugins/soundclo...
Name: ScriptPath, Length: 1000, dtype: object
31000    data/handsontable/handsontable/test/jasmine/sp...
31001    data/sidorares/node-x11/examples/vncviewer/unp...
31002    data/zohooo/jaxedit/library/mathjax/unpacked/j...
31003    data/wongiseng/SNEEditor/war/aida/ThesaurusSer...
31004              data/cujojs/rest/test/util/find-test.js
          

Name: ScriptPath, Length: 1000, dtype: object
57000    javascript-malware-collection-master\2016\2016...
57001    javascript-malware-collection-master\2016\2016...
57002    javascript-malware-collection-master\2016\2016...
57003    javascript-malware-collection-master\2016\2016...
57004    javascript-malware-collection-master\2016\2016...
                               ...                        
57995    javascript-malware-collection-master\2016\2016...
57996    javascript-malware-collection-master\2016\2016...
57997    javascript-malware-collection-master\2016\2016...
57998    javascript-malware-collection-master\2016\2016...
57999    javascript-malware-collection-master\2016\2016...
Name: ScriptPath, Length: 1000, dtype: object
58000    javascript-malware-collection-master\2016\2016...
58001    javascript-malware-collection-master\2016\2016...
58002    javascript-malware-collection-master\2016\2016...
58003    javascript-malware-collection-master\2016\2016...
58004    javascript-mal

In [121]:
for i in d:
    print(i.shape)

(79425, 2)


In [106]:
chunksize = 10 ** 6
with pd.read_csv('G:\JSContanaDataSet\XSS_dataset.csv', chunksize=20) as reader:
    for chunk in reader:
        process(chunk)

AttributeError: __enter__

In [103]:
chunksize = 20
with pd.read_csv(filepath_or_buffer = 'G:\JSContanaDataSet\XSS_dataset.csv',iterator=True,chunksize=chunksize) as reader:
    for chunk in reader:
        process(chunk)

AttributeError: __enter__

In [101]:
d.read(2)

,ScriptPath,Label
60,data/mrngoitall/nominatr/config/env/all.js,0
61,data/cdnjs/cdnjs/ajax/libs/backbone.marionette...,0
62,data/cdnjs/cdnjs/ajax/libs/yui/3.8.0/attribute...,0
63,data/Juniper/contrail-web-core/src/serverroot/...,0
64,data/mcongrove/ChariTi/modules/commonjs/ti.clo...,0
...,...,...
79420,javascript-malware-collection-master\2017\2017...,1
79421,javascript-malware-collection-master\2017\2017...,1
79422,javascript-malware-collection-master\2017\2017...,1
79423,javascript-malware-collection-master\2017\2017...,1


In [42]:
print(train_df.shape)
print(train_df[train_df['Label']==0].count()[1])
print(train_df[train_df['Label']==1].count()[1])
train_df.head()

(300, 2)
148
152


,ScriptPath,Label
50907,javascript-malware-collection-master\2016\2016...,1
4711,data/mindrevolution/SirTrevor-for-Umbraco/src/...,0
71602,javascript-malware-collection-master\2016\2016...,1
13199,data/ariatemplates/ariatemplates/test/aria/tem...,0
75627,javascript-malware-collection-master\2016\2016...,1


In [64]:
train_batch_size = 50
test_batch_size = int(test_df.shape[0]/(train_df.shape[0]/train_batch_size))
validate_batch_size = int(validation_df.shape[0]/(train_df.shape[0]/train_batch_size))

In [65]:
print(int(train_batch_size))
print(int(test_batch_size))
print(int(validate_batch_size))

50
16
16


In [66]:
print(int(500/train_batch_size))
print(int(100/test_batch_size))
print(int(100/validate_batch_size))

10
6
6


In [47]:
train_batch_size = 50
test_batch_size = int(test_df.shape[0]/(train_df.shape[0]*train_batch_size))
validate_batch_size = int(validation_df.shape[0]/(train_df.shape[0]*train_batch_size))
# num_words = 400000
num_words = 400
# maxlen = 1024
maxlen = 100
embedding_dim = 50
myoptimizer = 'adam'
myloss= 'binary_crossentropy'
mymetrics = 'accuracy'

traingen = CustomDataGen(train_df,
                         X_col={'path':'ScriptPath'},
                         y_col={'name': 'Label'},
                         batch_size=train_batch_size,
                         parent_path='G:\JSContanaDataSet',
                         fit=True)
save_object(traingen, "traingen_CustomDataGen")
validategen = CustomDataGen(validation_df,
                         X_col={'path':'ScriptPath'},
                         y_col={'name': 'Label'},
                         batch_size=validate_batch_size,
                         parent_path='G:\JSContanaDataSet',
                         fit=False)
save_object(validategen, "validategen_CustomDataGen")
testgen = CustomDataGen(test_df,
                         X_col={'path':'ScriptPath'},
                         y_col={'name': 'Label'},
                         batch_size=test_batch_size,
                         parent_path='G:\JSContanaDataSet',
                         fit=False)
save_object(testgen, "testgen_CustomDataGen")
# X_train , y_train = traingen.__getitem__(0)
# print(X_train)
# X_train_chunks = [] 
# y_train_chunks = []

# X_train_chunks.append(X_train)
# y_train_chunks.append(y_train)

# for i in range(1,int(train_df.shape[0]/train_batch_size)):
#     X_train_chunks.append(traingen.__getitem__(i)[0])
#     y_train_chunks.append(traingen.__getitem__(i)[1])
# save_object(X_train_chunks, "X_train_chunks")
# save_object(y_train_chunks, "y_train_chunks")


# X_test , y_test = testgen.__getitem__(0)
# print(X_test)
# X_test_chunks = [] 
# y_test_chunks = []

# X_test_chunks.append(X_test)
# y_test_chunks.append(y_test)

# for i in range(1,int(test_df.shape[0]/test_batch_size)):
#     X_test_chunks.append(testgen.__getitem__(i)[0])
#     y_test_chunks.append(testgen.__getitem__(i)[1])
# save_object(X_test_chunks, "X_test_chunks")
# save_object(y_test_chunks, "y_test_chunks")

# X_validate , y_validate = validategen.__getitem__(0)
# print(X_validate)
# X_validate_chunks = [] 
# y_validate_chunks = []

# X_validate_chunks.append(X_validate)
# y_validate_chunks.append(y_validate)

# for i in range(1,int(validation_df.shape[0]/validate_batch_size)):
#     X_validate_chunks.append(validategen.__getitem__(i)[0])
#     y_validate_chunks.append(validategen.__getitem__(i)[1])
# save_object(X_validate_chunks, "X_validate_chunks")
# save_object(y_validate_chunks, "y_validate_chunks")
# vocab_size = load_object('vocab_size')
# vocab_size

In [49]:
# X_validate_chunks[0]

In [ ]:
def creat_metrics():
    model_comparison_table = {}
    model_comparison_table['chunk_nuber'] = []
    model_comparison_table['train_loss'] = []
    model_comparison_table['train_accuracy'] = []
    model_comparison_table['test_loss'] = []
    model_comparison_table['test_accuracy'] = []
    model_comparison_table['classification_report'] = []
    return model_comparison_table

In [ ]:
# model_comparison_table = {}
# model_comparison_table['chunk_nuber'] = []
# model_comparison_table['train_loss'] = []
# model_comparison_table['train_accuracy'] = []
# model_comparison_table['test_loss'] = []
# model_comparison_table['test_accuracy'] = []
# model_comparison_table['classification_report'] = []

In [50]:
int(test_df.shape[0]/test_batch_size)

6

In [22]:
def inputer(model_comparison_table,chunk_nuber,train_loss, train_accuracy,test_loss,test_accuracy,classification_report):#precision_score,recall_score,f1_score):
    model_comparison_table['chunk_nuber'].append(str(chunk_nuber))
    model_comparison_table['train_loss'].append(train_loss)
    model_comparison_table['train_accuracy'].append(train_accuracy)
    model_comparison_table['test_loss'].append(test_loss)    
    model_comparison_table['test_accuracy'].append(test_accuracy)  
    model_comparison_table['classification_report'].append(classification_report)    

In [1]:
model_comparison_table = creat_metrics()
train_test_Model = create_model()
train_test_Model.save("train_test_Model.h5")
for i in range(int(test_df.shape[0]/test_batch_size)):
    print("================ "+str(i)+" ================")
    train_test_Model = load_model("train_test_Model.h5")
    X_train,y_train = X_train_chunks[i],y_train_chunks[i]
    X_validation,y_validation = X_test_chunks[i],y_test_chunks[i]
    train_test_Model , history_train_test_Model = fit_model(train_test_Model,X_train,y_train,X_validation,y_validation)
    evaluation(train_test_Model,i,history_train_test_Model,X_train,y_train,X_validation,y_validation,model_comparison_table)
    train_test_Model.save("train_test_Model.h5")
save_object(train_test_Model, "train_test_Model")
save_object(history_train_test_Model, "history_train_test_Model")
save_object(model_comparison_table, "model_comparison_table")


In [ ]:
df = pd.DataFrame(model_comparison_table)
df.to_csv('G:/JSContanaDataSet/result_df.csv',index=False)
df.head(5)

In [ ]:
keras_function = K.function([train_test_Model.input], [train_test_Model.layers[1].output])
for_traingen = []
for i in range(len(X_train_chunks)):
    first_layer_output = keras_function([X_train_chunks[i], 1])
    for_traingen.append(first_layer_output[0])
save_object(for_traingen, "for_traingen")
# for_traingen = load_object("for_traingen")

In [ ]:
# model_comparison_table_1 = {}
# model_comparison_table_1['chunk_nuber'] = []
# model_comparison_table_1['train_loss'] = []
# model_comparison_table_1['train_accuracy'] = []
# model_comparison_table_1['test_loss'] = []
# model_comparison_table_1['test_accuracy'] = []
# model_comparison_table_1['classification_report'] = []

In [ ]:
model_comparison_table_1 = creat_metrics()
validate_test_Model = create_model()
validate_test_Model.save("validate_test_Model.h5")
for i in range(len(X_validate_chunks)):
    print("================ "+str(i)+" ================")
    validate_test_Model = load_model("validate_test_Model.h5")
    X_train,y_train = X_validate_chunks[i],y_validate_chunks[i]
    X_validation,y_validation = X_test_chunks[i],y_test_chunks[i]
    validate_test_Model , history_validate_test_Model = fit_model(validate_test_Model,X_train,y_train,X_validation,y_validation)
    evaluation(validate_test_Model,history_validate_test_Model,X_train,y_train,X_validation,y_validation,model_comparison_table_1)
    validate_test_Model.save("validate_test_Model.h5")
save_object(validate_test_Model, "validate_test_Model")
save_object(history_validate_test_Model, "history_validate_test_Model")
save_object(model_comparison_table_1, "model_comparison_table_1")

In [ ]:
save_object(model_comparison_table_1, "model_comparison_table_1")
df_1 = pd.DataFrame(model_comparison_table_1)
df_1.to_csv('G:/JSContanaDataSet/result_df_1.csv',index=False)
df_1.head(5)

In [ ]:
keras_function = K.function([validate_test_Model.input], [validate_test_Model.layers[1].output])
for_validate = []
for i in range(len(X_validate_chunks)):
    first_layer_output = keras_function([X_validate_chunks[i], 1])
    for_validate.append(first_layer_output[0])
save_object(for_validate, "for_validate")
# for_validate = load_object("for_validate")

In [ ]:
# model_comparison_table_2 = {}
# model_comparison_table_2['chunk_nuber'] = []
# model_comparison_table_2['train_loss'] = []
# model_comparison_table_2['train_accuracy'] = []
# model_comparison_table_2['test_loss'] = []
# model_comparison_table_2['test_accuracy'] = []
# model_comparison_table_2['classification_report'] = []

In [ ]:
model_comparison_table_2 = creat_metrics()
lstm_Model = create_model(shape=for_traingen[0][0].shape,without_lstm=True)
lstm_Model.save("lstm_Model.h5")
for i in range(len(for_traingen)):
    print("================ "+str(i)+" ================")
    lstm_Model = load_model("lstm_Model.h5")
    X_train,y_train = feature_selection(for_traingen[i],y_train_chunks[i])
    X_validation,y_validation = feature_selection(for_validate[i],y_validate_chunks[i])
    lstm_Model , history_lstm_Model = fit_model(lstm_Model,X_train,y_train,X_validation,y_validation)
    evaluation(lstm_Model,history_lstm_Model,X_train,y_train,X_validation,y_validation,model_comparison_table_2)
    lstm_Model.save("lstm_Model.h5")
save_object(lstm_Model, "lstm_Model")
save_object(history_lstm_Model, "history_lstm_Model")
save_object(model_comparison_table_2, "model_comparison_table_2")

In [ ]:
save_object(model_comparison_table_2, "model_comparison_table_2")
df_2 = pd.DataFrame(model_comparison_table_2)
df_2.to_csv('G:/JSContanaDataSet/result_df_2.csv',index=False)
df_2.head(5)

In [ ]:
HL_model = Model(inputs=model.input, outputs=model.layers[7].output)

In [ ]:
DL_model = Model(inputs=model.layers[8].input, outputs=model.layers[-1].output)

In [ ]:
# model_comparison_table_3 = {}
# model_comparison_table_3['chunk_nuber'] = []
# model_comparison_table_3['train_loss'] = []
# model_comparison_table_3['train_accuracy'] = []
# model_comparison_table_3['test_loss'] = []
# model_comparison_table_3['test_accuracy'] = []
# model_comparison_table_3['classification_report'] = []

In [ ]:
model_comparison_table_3 = creat_metrics()
lstm_Model = create_model(shape=for_traingen[0][0].shape,without_lstm=True)
lstm_Model.save("lstm_Model.h5")
for i in range(len(for_traingen)):
    print("================ "+str(i)+" ================")
    lstm_Model = load_model("lstm_Model.h5")
    X_train,y_train = feature_selection(for_traingen[i],y_train_chunks[i])
    X_validation = feature_selection_(for_validate[i])
    y_validation = y_validate_chunks[i]
    lstm_Model , history_lstm_Model = fit_model(lstm_Model,X_train,y_train,X_validation,y_validation)
    evaluation(lstm_Model,history_lstm_Model,X_train,y_train,X_validation,y_validation,model_comparison_table_3)
    lstm_Model.save("lstm_Model.h5")


In [ ]:
save_object(model_comparison_table_3, "model_comparison_table_3")
df_3 = pd.DataFrame(model_comparison_table_3)
df_3.to_csv('G:/JSContanaDataSet/result_df_3.csv',index=False)
df_3.head(5)

In [ ]:
df_1 = pd.read_csv('G:/JSContanaDataSet/result_df.csv')

In [ ]:
df_2 = pd.read_csv('G:/JSContanaDataSet/result_df_3.csv')

In [ ]:
df_1.head(5)

In [ ]:
df_2.head(5)

In [ ]:
for i in range(5):
    print(df_1['classification_report'][i])
    print(df_2['classification_report'][i])
    print("====================================================================")